# Spark DataFrame Basics

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
df = spark.read.json('people.json')

In [4]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
df.printSchema()  # To get the schema

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
df.columns  #Attribute - as we dont need to have paranthesis there

['age', 'name']

In [7]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [9]:
df.describe().show() #To view the describe dataframe - gives statistical summary of the numeric column

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [10]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [11]:
data_schema = [StructField('age', IntegerType(), True),
              StructField('name', StringType(), True)]

# Name of the column, which type you want it to be, True- if you are okay with it having null values

In [12]:
final_struc = StructType(fields = data_schema)

In [13]:
df = spark.read.json('people.json', schema = final_struc)

In [14]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [15]:
print(df['age'])
print(type(df['age']))

Column<b'age'>
<class 'pyspark.sql.column.Column'>


In [16]:
df.select('age')

DataFrame[age: int]

In [17]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [18]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [19]:
df.head(2)[0]

Row(age=None, name='Michael')

In [20]:
# To add/replace a column
df.withColumn('newage', df['age'])

DataFrame[age: int, name: string, newage: int]

In [21]:
df.withColumn('newage', df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [22]:
#To rename a column
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [23]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [26]:
# To connect to SQL table
df.createOrReplaceTempView('people')
results = spark.sql('SELECT * FROM people')
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



# Spark DataFrame Basic Operations

In [28]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ops').getOrCreate()
df = spark.read.csv('appl_stock.csv', inferSchema = True, header = True)

In [29]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [32]:
# Spark dataframe are built on spark sql platform
# SQL WAY:
df.filter("Close<200").show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-22|206.78000600000001|        207.499996|            197.16|            197.75|220441900|         25.620401|
|2010-01-28|        204.930004|        205.500004|        198.699995|        199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|        202.199995|        190.250002|        192.060003|311488100|         24.883208|
|2010-02-01|192.36999699999998|             196.0|191.29999899999999|        194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|        196.319994|193.37999299999998|        195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|        200.200003|      

In [33]:
df.filter("Close<200").select('Open').show()

+------------------+
|              Open|
+------------------+
|206.78000600000001|
|        204.930004|
|        201.079996|
|192.36999699999998|
|        195.909998|
|        195.169994|
|        196.730003|
|192.63000300000002|
|        195.690006|
|        196.419996|
|        195.889997|
|        194.880001|
|        199.999998|
|         92.699997|
|         94.730003|
|         94.129997|
|         94.040001|
|         92.199997|
|         91.510002|
|         92.309998|
+------------------+
only showing top 20 rows



In [34]:
# PYTHON WAY
#df.filter("Close<200").show()
df.filter(df['Close']<200).show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-22|206.78000600000001|        207.499996|            197.16|            197.75|220441900|         25.620401|
|2010-01-28|        204.930004|        205.500004|        198.699995|        199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|        202.199995|        190.250002|        192.060003|311488100|         24.883208|
|2010-02-01|192.36999699999998|             196.0|191.29999899999999|        194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|        196.319994|193.37999299999998|        195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|        200.200003|      

In [35]:
df.filter((df['Open']>200) & (df['Close']<200)).show()
# for not operator, its ~
# df.filter((df['Open']>200) & ~(df['Close']<200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [36]:
# To work on the data, we can actually use collect 
result = df.filter(df['Low']==197.16).collect()
result

[Row(Date='2010-01-22', Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

In [37]:
result[0].asDict()['Open']

206.78000600000001

# Groupby and Aggregate

In [38]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ops').getOrCreate()
df = spark.read.csv('sales_info.csv', inferSchema = True, header = True)

In [39]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [40]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   GOOG|            220.0|
|   MSFT|322.3333333333333|
|     FB|            610.0|
|   APPL|            370.0|
+-------+-----------------+



In [42]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [44]:
df.groupBy('Company').agg({'Sales':'sum'}).show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   GOOG|     660.0|
|   MSFT|     967.0|
|     FB|    1220.0|
|   APPL|    1480.0|
+-------+----------+



In [53]:
from pyspark.sql.functions import countDistinct, avg, stddev, format_number
df.select(avg('Sales').alias('Avg_sales')).select(format_number('Avg_sales', 2).alias('new_avg')).show()
# temp = df.select(avg('Sales').alias('Avg_sales'))
# temp.select(format_number('Avg_sales', 2)).show()

+-------+
|new_avg|
+-------+
| 360.58|
+-------+



In [54]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [55]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



# Dealing with missing data

In [56]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('miss').getOrCreate()
df = spark.read.csv('ContainsNull.csv', inferSchema=True, header=True)

In [57]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [58]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [59]:
# Need to have atleast 2 non null value to not get dropped
df.na.drop(thresh = 2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [60]:
df.na.drop(how = 'all').show()  #by default its any. 'all' --drop only if all the rows are null

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [61]:
df.na.drop(subset = ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [62]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [63]:
df.na.fill('BLANK').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2|BLANK| null|
|emp3|BLANK|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [64]:
df.na.fill('NO NAME', subset = ['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|NO NAME| null|
|emp3|NO NAME|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [65]:
from pyspark.sql.functions import mean
mean_val =df.select(mean(df['Sales'])).collect()[0][0]
df.na.fill(mean_val, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



# Date and Time

In [66]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('time').getOrCreate()
df = spark.read.csv('appl_stock.csv', inferSchema = True, header = True)

In [69]:
from pyspark.sql.functions import dayofmonth, hour, dayofyear, month, year, format_number, date_format

In [70]:
df.select(month(df['Date'])).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+
only showing top 20 rows



In [78]:
new_df = df.withColumn("Year", year(df['Date']))
result= new_df.groupBy("Year").mean().select(['Year', "avg(Close)"])
result.withColumnRenamed("avg(Close)", "AVERAGE").select(['Year',format_number('AVERAGE', 2).alias("AVG")]).show()


+----+------+
|Year|   AVG|
+----+------+
|2010|259.84|
|2011|364.00|
|2012|576.05|
|2013|472.63|
|2014|295.40|
|2015|120.04|
|2016|104.60|
+----+------+

